In [ ]:
# Install required libraries
%pip install transformers>=4.41 accelerate datasets sentence-transformers


In [ ]:
# Import necessary modules
from datasets import load_dataset


In [ ]:
# Load the rotten_tomatoes dataset
dataset = load_dataset("rotten_tomatoes")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [ ]:
# Display the DatasetDict structure
print(dataset)
print(f"\nTrain: {len(dataset['train'])} rows")
print(f"Validation: {len(dataset['validation'])} rows")
print(f"Test: {len(dataset['test'])} rows")


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

Train: 8530 rows
Validation: 1066 rows
Test: 1066 rows


## Using a task-specific model


In [ ]:
from transformers import pipeline
from tqdm import tqdm
from sklearn.metrics import classification_report


In [ ]:
classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

predictions = []
for text in tqdm(test_texts, desc="Running inference"):
    result = classifier(text)[0]
    score = result["score"]
    label = result["label"].lower()

    if "positive" in label:
        pred = 1
    else:
        pred = 0

    predictions.append(pred)



Running inference:   1%|          | 10/1066 [00:01<01:42, 10.26it/s]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset

Running inference: 100%|██████████| 1066/1066 [00:12<00:00, 87.30it/s] 


In [ ]:
print(classification_report(test_labels, predictions, target_names=["negative", "positive"]))


              precision    recall  f1-score   support

    negative       0.68      0.94      0.79       533
    positive       0.91      0.56      0.69       533

    accuracy                           0.75      1066
   macro avg       0.79      0.75      0.74      1066
weighted avg       0.79      0.75      0.74      1066



## Classification tasks with embeddings


### A) Supervised classification with embeddings


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np


In [ ]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
train_texts = dataset["train"]["text"]
train_labels = dataset["train"]["label"]
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

train_embeddings = model.encode(train_texts, show_progress_bar=True)
test_embeddings = model.encode(test_texts, show_progress_bar=True)


Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [ ]:
classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_embeddings, train_labels)
predictions = classifier.predict(test_embeddings)


In [ ]:
print(classification_report(test_labels, predictions, target_names=["negative", "positive"]))


              precision    recall  f1-score   support

    negative       0.85      0.86      0.85       533
    positive       0.86      0.85      0.85       533

    accuracy                           0.85      1066
   macro avg       0.85      0.85      0.85      1066
weighted avg       0.85      0.85      0.85      1066



### B) Unsupervised use case


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

train_embeddings_neg = train_embeddings[train_labels == 0]
train_embeddings_pos = train_embeddings[train_labels == 1]

class_centroid_neg = train_embeddings_neg.mean(axis=0)
class_centroid_pos = train_embeddings_pos.mean(axis=0)

class_centroid_neg = np.nan_to_num(class_centroid_neg, nan=0.0)
class_centroid_pos = np.nan_to_num(class_centroid_pos, nan=0.0)


/tmp/ipython-input-1534935087.py:6: RuntimeWarning: Mean of empty slice.
  class_centroid_neg = train_embeddings_neg.mean(axis=0)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/tmp/ipython-input-1534935087.py:7: RuntimeWarning: Mean of empty slice.
  class_centroid_pos = train_embeddings_pos.mean(axis=0)


In [ ]:
centroids = np.vstack([class_centroid_neg, class_centroid_pos])
test_embeddings_clean = np.nan_to_num(test_embeddings, nan=0.0)
similarities = cosine_similarity(test_embeddings_clean, centroids)
predictions = similarities.argmax(axis=1)


In [ ]:
print(classification_report(test_labels, predictions, target_names=["negative", "positive"]))


              precision    recall  f1-score   support

    negative       0.50      1.00      0.67       533
    positive       0.00      0.00      0.00       533

    accuracy                           0.50      1066
   macro avg       0.25      0.50      0.33      1066
weighted avg       0.25      0.50      0.33      1066



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### C) Zero-shot classification


In [ ]:
label_descriptions = ["A negative review", "A positive review"]
label_embeddings = model.encode(label_descriptions)


In [ ]:
similarities = cosine_similarity(test_embeddings, label_embeddings)
predictions = similarities.argmax(axis=1)


In [ ]:
print(classification_report(test_labels, predictions, target_names=["negative", "positive"]))


              precision    recall  f1-score   support

    negative       0.78      0.77      0.78       533
    positive       0.77      0.79      0.78       533

    accuracy                           0.78      1066
   macro avg       0.78      0.78      0.78      1066
weighted avg       0.78      0.78      0.78      1066



## Text classification with generative models


In [32]:
%pip install groq python-dotenv


In [33]:
import os

# Try to get API key from Colab secrets first
try:
    from google.colab import userdata
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
except:
    pass

# Check if API key is set
if not os.environ.get("GROQ_API_KEY"):
    raise ValueError("GROQ_API_KEY not found. Please set it using Colab secrets (recommended) or uncomment the line in the cell above.")


In [34]:
from groq import Groq
from tqdm import tqdm
from sklearn.metrics import classification_report

if not os.environ.get("GROQ_API_KEY") or os.environ["GROQ_API_KEY"] == "":
    raise ValueError("GROQ_API_KEY is not set. Please configure it in the previous cell.")

client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [35]:
def groq_predict(text):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a sentiment classifier. Respond with 'positive' or 'negative' only."},
            {"role": "user", "content": f"Classify the sentiment of this movie review: {text}"}
        ],
        temperature=0.1,
        max_tokens=10
    )
    prediction = response.choices[0].message.content.strip().lower()
    if "positive" in prediction:
        return "positive"
    else:
        return "negative"


In [36]:
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

predictions = []
for text in tqdm(test_texts, desc="Runnin+g Groq predictions"):
    pred = groq_predict(text)
    predictions.append(pred)


Running Groq predictions: 100%|██████████| 1066/1066 [37:54<00:00,  2.13s/it]


In [37]:
predictions_mapped = [1 if p == "positive" else 0 for p in predictions]
print(classification_report(test_labels, predictions_mapped, target_names=["negative", "positive"]))


              precision    recall  f1-score   support

    negative       0.86      0.96      0.91       533
    positive       0.95      0.85      0.90       533

    accuracy                           0.90      1066
   macro avg       0.91      0.90      0.90      1066
weighted avg       0.91      0.90      0.90      1066



## Text2Text transfert transformers


In [38]:
import torch
from transformers import pipeline
from tqdm import tqdm
from sklearn.metrics import classification_report

device = "cuda:0" if torch.cuda.is_available() else "cpu"
t5_pipeline = pipeline("text2text-generation", model="google/flan-t5-small", device=device)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [39]:
def add_t5_prompt(example):
    prompt = "Is the following sentence positive or negative? "
    example["t5"] = prompt + example["text"]
    return example

dataset = dataset.map(add_t5_prompt)


Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [40]:
test_t5 = dataset["test"]["t5"]
test_labels = dataset["test"]["label"]

predictions = []
for text in tqdm(test_t5, desc="Running T5 inference"):
    result = t5_pipeline(text, max_length=10)[0]["generated_text"]
    predictions.append(result.strip().lower())


Running T5 inference:   0%|          | 1/1066 [00:00<15:59,  1.11it/s]Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Running T5 inference:   0%|          | 3/1066 [00:01<04:50,  3.66it/s]Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. 

In [42]:
predictions_mapped = []
for pred in predictions:
    if "positive" in pred:
        predictions_mapped.append(1)
    else:
        predictions_mapped.append(0)

print(classification_report(test_labels, predictions_mapped, target_names=["negative", "positive"]))


              precision    recall  f1-score   support

    negative       0.83      0.85      0.84       533
    positive       0.85      0.83      0.84       533

    accuracy                           0.84      1066
   macro avg       0.84      0.84      0.84      1066
weighted avg       0.84      0.84      0.84      1066



## Conclusion

This lab explored multiple approaches to text classification, demonstrating the trade-offs between different methods. Task-specific models like RoBERTa fine-tuned for sentiment analysis typically achieve the highest performance by leveraging domain-specific training. Supervised classification with embeddings offers a flexible alternative that combines pre-trained representations with task-specific classifiers. Unsupervised centroid-based methods provide a simple baseline without requiring labeled data, while zero-shot classification with label descriptions enables classification without training examples. Generative LLMs via API calls offer flexibility and ease of use but may be slower and more expensive. Text2Text transformers like T5 demonstrate how sequence-to-sequence models can be adapted for classification tasks through prompting. The choice of method depends on factors such as available training data, computational resources, latency requirements, and desired accuracy.
